# load All Required Packages

In [33]:
import  pandas  as  pd
import os
import pandas as pd
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np 
from   scipy.optimize import minimize

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook_connected"

# Read Files For Specified Path

In [34]:
# directory containing the CSV files
path1 = '../../Dataset/project/All_homes_prices.csv'
path2 = '../../Dataset/project/Rental.csv'
path3 = '../../Dataset/project/Sales_Price.csv'


In [35]:
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)
df3 = pd.read_csv(path3)

In [36]:
df1.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28
0,102001,0,United States,country,NaN,110923.198270,111118.282669,111360.068873,111878.535148,112475.107793,...,324744.159760,329342.041666,331792.218812,332259.177531,331163.558363,330120.694680,329278.758652,328198.452385,327569.041698,327514.193013
1,394913,1,"New York, NY",msa,NY,177576.648285,178332.354083,179093.887620,180636.222410,182230.307302,...,555964.194958,561981.143170,565974.450796,567041.525519,565252.038000,563198.956329,561477.222533,559150.425251,559116.909909,560268.482672
2,753899,2,"Los Angeles, CA",msa,CA,220312.767961,221127.682496,222216.500981,224401.746656,226799.131307,...,894278.194789,900738.474685,902446.881365,899148.225112,893257.250064,887956.929337,884296.450518,880012.047421,871774.871645,861761.432293
3,394463,3,"Chicago, IL",msa,IL,136730.155085,136859.794881,137103.759555,137711.616205,138441.512620,...,281110.836812,283000.621117,283401.864739,282776.279977,281211.686384,279965.348367,279297.880845,278649.392155,279202.085743,280231.051678
4,394514,4,"Dallas, TX",msa,TX,126826.643490,126890.108652,126961.829934,127141.253155,127370.473737,...,368251.484003,373721.718566,376467.582907,376495.453581,374561.804693,372512.410198,370635.750644,368144.107192,365604.408379,363545.511501


# Set UP functions for further cleaning and merging

In [37]:
def clean(df):
    # remove rows with missing values
    df = df.fillna(df.mean())
    df = df.drop(index=0)
    # fill in missing values with mean of each column
    

    #df['RegionName'] = df['RegionName'].str.split(',').str[0]
    return df

In [38]:
''' 
Takes in a dataset, return a a dictionary with the key is the Region Name, the value is a dataset of that region
'''
def group_state(df):
    # group DataFrame by state column
    grouped = df.groupby('RegionName')

    df = dict()
    # iterate over groups and print data for each state
    for state, data in grouped:
        df[state] = data

    return df

In [39]:
''' 
Takes in a dictionary of datasets. Returns a list of the datasets with column changes: drop unused, rename index, calculate mean for NUll values.
'''
def transform(df_dict):
    for i in df_dict:
        state = []
        df_dict[i] = pd.DataFrame(df_dict[i].mean(), columns=['mean'])
        df_dict[i] = df_dict[i].reset_index()
        df_dict[i] = df_dict[i].rename(columns={'index': 'date'})
        df_dict[i] = df_dict[i].drop(index=0)
        df_dict[i] = df_dict[i].drop(index=1)

        for j in range(df_dict[i].shape[0]):
            state.append(i)
        df_dict[i]['state'] = state

    t = []
    for i in df_dict:
        t.append(df_dict[i])
    return t

In [40]:
df1 = clean(df1)
df2 = clean(df2)
df3 = clean(df3)

In [41]:
d1 = group_state(df1)
d2 = group_state(df2)
d3 = group_state(df3)

In [43]:
df2.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,...,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28
1,394913,1,"New York, NY",msa,NY,2386.585780,2405.273501,2422.713710,2440.732457,2449.784467,...,3017.668773,3079.743085,3133.665788,3157.542046,3154.523579,3131.359562,3099.155451,3075.999271,3068.361445,3083.935335
2,753899,2,"Los Angeles, CA",msa,CA,1918.511039,1931.679906,1945.134903,1956.521807,1972.651475,...,2863.225552,2888.255092,2909.795304,2923.254438,2930.609295,2925.944799,2917.848297,2909.862264,2904.846076,2907.943492
3,394463,3,"Chicago, IL",msa,IL,1427.222929,1440.010040,1451.347989,1475.071800,1481.594637,...,1825.495290,1846.851392,1866.064802,1874.456008,1877.134926,1875.555990,1869.430226,1870.250969,1873.910478,1888.474027
4,394514,4,"Dallas, TX",msa,TX,1148.814439,1159.221531,1171.106575,1178.902425,1185.122950,...,1754.977334,1789.078314,1813.648014,1824.177590,1823.576538,1811.405487,1796.557207,1781.243863,1775.729839,1778.671326
5,394692,5,"Houston, TX",msa,TX,1280.894885,1288.877292,1297.907743,1306.981089,1311.971157,...,1620.224031,1634.995323,1647.702017,1654.945314,1654.160553,1649.009218,1640.600126,1635.055768,1640.477920,1648.400288


In [28]:
t1 = transform(d1)
t2 = transform(d2)
t3 = transform(d3)

TypeError: Could not convert ['Aberdeen, SD' 'msa' 'SD'] to numeric

In [ ]:
t3[0].head()

,date,mean,state
2,2018-08-31,146080.0,"Abilene, TX"
3,2018-09-30,148723.0,"Abilene, TX"
4,2018-10-31,159568.0,"Abilene, TX"
5,2018-11-30,164747.0,"Abilene, TX"
6,2018-12-31,167301.0,"Abilene, TX"


# Combine the dataset for each category: Home value, Sale price, Rental price

In [ ]:
Home_value = pd.concat(t1, axis=0)
Rental = pd.concat(t2, axis=0)
Sale = pd.concat(t3, axis=0)

# Rename Columns

In [ ]:
Home_value = Home_value.rename(columns={'mean': 'Mean Home Value'})
Rental = Rental.rename(columns={'mean': 'Mean Rental Price'})
Sale = Sale.rename(columns={'mean': 'Mean Sale Price'})

In [ ]:
Sale.head()

,date,Mean Sale Price,state
2,2018-08-31,146080.0,"Abilene, TX"
3,2018-09-30,148723.0,"Abilene, TX"
4,2018-10-31,159568.0,"Abilene, TX"
5,2018-11-30,164747.0,"Abilene, TX"
6,2018-12-31,167301.0,"Abilene, TX"


# Set Up the theme of the views, this will be different based on views.

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
import matplotlib as mpl
from pathlib import Path
cfgdir = mpl.get_configdir() # find your configuration folder
p = Path(cfgdir)
print(p)
stylelib = (p / 'stylelib')
stylelib.mkdir(exist_ok=True)
path = stylelib / 'project.mplstyle' # create paths
path.write_text(''' # write into the file
axes.facecolor : F8F3F9
font.family : Arial
lines.linewidth : 5
xtick.labelsize : 16
ytick.labelsize : 16
axes.grid : True
grid.color : E0DDE1
grid.linestyle: dotted
grid.linewidth: 1.2
lines.linestyle : dotted
axes.titlesize : 23

lines.markersize : 14
axes.edgecolor : black
axes.linewidth : 2
axes.labelsize : 15
axes.labelcolor : black
axes.labelweight : normal
legend.title_fontsize : 16
axes.prop_cycle: cycler('color', ['9355AA', '6CAA55'])
figure.dpi : 200
''')

# LOAD STYLE 
plt.style.reload_library()
print('project' in plt.style.available)
# AFTER 
plt.style.use([ 'project'])

C:\Users\yzh20\.matplotlib
True


In [ ]:
plt.rcParams.update(plt.rcParamsDefault)

Merged together

In [ ]:
# join the dataframes based on columns
df2 = pd.merge(Sale, Home_value, on=['date', 'state'])
df3 = pd.merge(df2, Rental, on=['date', 'state'])

In [ ]:
df3.head()

,date,Mean Sale Price,state,Mean Home Value,Mean Rental Price
0,2018-08-31,146080.0,"Abilene, TX",120797.789890,764.209329
1,2018-09-30,148723.0,"Abilene, TX",121574.982835,761.977838
2,2018-10-31,159568.0,"Abilene, TX",122212.240182,759.308050
3,2018-11-30,164747.0,"Abilene, TX",122799.685424,763.234543
4,2018-12-31,167301.0,"Abilene, TX",123239.513824,767.126959


In [ ]:
# join the dataframes based on columns
df2t = pd.merge(Sale, Home_value, on=['date', 'state'])
df3t = pd.merge(df2, Rental, on=['date', 'state'])

In [ ]:
df3t.to_csv('../../Dataset/Cleaned/Merged.csv', index=False)

# Make Selections of the data

In [79]:
def selection(df_dict):
    dfs = []
    for i in df_dict:
        if i == 'San Jose, CA' or i == 'Santa Cruz, CA' or i == 'Breckenridge, CO' or i == 'Kahului, HI' or i == 'Edwards, CO'\
             or i == 'Napa, CA' or i =='Los Angeles, CA' or i =='Key West, FL' or i =='Salinas, CA' or i =='San Diego, CA'\
             or i =='San Luis Obispo, CA' or i =='Oxnard, CA' or i =='Santa Maria, CA' or i =='Santa Rosa, CA' or i =='Boulder, CO' \
            or i =='Urban Honolulu, HI' or i =='Seattle, WA' or i =='Bend, OR' or i =='Kalispell, MT' or i =='Truckee, CA':
            dfs.append(df_dict[i])

    df = pd.concat(dfs, axis=0)
    return df

In [80]:
Homevalue_selected = selection(d1)
Rental_selected = selection(d2)
Sale_selected = selection(d3)

In [81]:
# join the dataframes based on columns
df2_t = pd.merge(Sale_selected, Homevalue_selected, on=['date', 'state'])
df_selected = pd.merge(df2, Rental_selected, on=['date', 'state'])

In [82]:
df_selected.head()

,date,Mean Sale Price,state,Mean Home Value,mean
0,2018-08-31,394588.0,"Bend, OR",381885.667338,1489.368110
1,2018-09-30,396351.0,"Bend, OR",383510.750632,1495.170848
2,2018-10-31,391015.0,"Bend, OR",384594.684964,1492.019209
3,2018-11-30,385500.0,"Bend, OR",385442.670086,1483.661094
4,2018-12-31,384032.0,"Bend, OR",387043.568806,1482.177364


In [83]:
df_selected['Metropolitan Area'] = df_selected['state']

In [85]:
df_selected.to_csv('../../data/Modified/Merged_downsample.csv', index=False)